# Библиотеки:

In [3]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

# Функции:

In [4]:
#Для извлечения каждого параметра написана небольшая функция

#С ценой все понятно, да и с остальными параметрами вроде тоже...
# Здесь мы берем 4 числа из списка с ценой, т.к. это позволит выкачивать цены большие миллиарда,
# а для цен меньше миллиарда это ничего не испортит

In [5]:
def getPrice(flat_page):
    try:
        price = flat_page.find('div', attrs={'class':'object_descr_price'})
        price = re.split('<div>|руб|\W', str(price))
        price = "".join([i for i in price if i.isdigit()][-4:])
        return int(price)
    except: 
        return np.NaN
    

In [6]:
#В качестве центра города взяты координаты Кремля
#Расстояние вычисляется по теореме Пифагора

In [7]:
lat_c = 55.752037
lon_c = 37.617778
def getDist(flat_page):
    try:
        coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
        coords = re.split('&amp|center=|%2C', str(coords))
        coords_list = []
        for item in coords:
            if item[0].isdigit():
                coords_list.append(item)
        lat = float(coords_list[0])
        lon = float(coords_list[1])
        return ((((lat_c - lat)**2)+((lon_c - lon)**2))**(1/2))/0.014723278711700902
    except: 
        return np.NaN   

In [8]:
#Дальше все более менее поянятно

In [9]:
def getRoom(flat_page):
    try:
        rooms = flat_page.find('div', attrs={'class':'object_descr_title'})
        rooms = html_stripper(rooms)
        room_number = ''
        for i in re.split('-|\n', rooms):
            if 'комн' in i:
                break
            else:
                room_number += i
        room_number = "".join(room_number.split())
        return room_number
    except: 
        return np.NaN 

In [10]:
# Дальше почти все берем из таблички на странице с квартирой

In [11]:
def getNew(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        new = re.split('Тип дома:|Тип продажи', table)[1]
        for i in re.split('-|\n', new):
            if 'вторичка' in i:
                return 0
            if 'новостройка' in i:
                return 1
        return np.NaN
    except: 
        return np.NaN 

In [12]:
def getTotsp(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        Totsp = re.split('Общая площадь:|Площадь комнат', table)[1]
        for i in re.split('-|\n', Totsp):
            if '\xa0м2' in i:
                a_0 = re.split('\xa0', i)[0]
                return int(re.split('-|,',a_0)[0])
        return np.NaN
    except: 
        return np.NaN 

In [13]:
def getKitsp(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        Kitsp = re.split('Площадь кухни:|санузлов', table)[1]
        for i in re.split('-|\n', Kitsp):
            if '\xa0м2' in i:
                a_0 = re.split('\xa0', i)[0]
                return int(re.split('-|,',a_0)[0])
        return np.NaN
    except: 
        return np.NaN 

In [14]:
def getLivesp(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        Livesp = re.split('Жилая площадь:|Площадь кухни', table)[1]
        for i in re.split('-|\n', Livesp):
            if '\xa0м2' in i:
                a_0 = re.split('\xa0', i)[0]
                return int(re.split('-|,',a_0)[0])
        return np.NaN
    except: 
        return np.NaN 

In [15]:
def getMetrdist(flat_page):  
    try:
        metro = flat_page.find('span', attrs = {'class':'object_item_metro_comment'})
        metro = html_stripper(metro)
        metro = re.split('-|\n', metro)
        if metro[1][len(metro[1])-2].isdigit() == True:
                return int(metro[1][len(metro[1])-2] + metro[1][len(metro[1])-1])
        else:
            return int(metro[1][len(metro[1])-1])
    except: 
        return np.NaN 

In [16]:
def getWalk(flat_page):
    try:
        metro = flat_page.find('span', attrs = {'class':'object_item_metro_comment'})
        metro = html_stripper(metro)
        metro = re.split('-|\n', metro)    
        for i in metro:
            if 'пешком' in i:
                return 1
        return 0
    except: 
        return np.NaN 

In [17]:
def getBrick(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        Brick = re.split('Тип дома:|Тип продажи', table)[1]
        for i in re.split('-|\n', Brick):
            i = i.replace(' ','')
            if ('кирпичный' in i) or ('монолитный' in i) or ('панельный' in i):
                return 1
        return 0
    except: 
        return np.NaN 

In [18]:
def getTel(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        Tel = re.split('Телефон:|Вид из окна', table)[1]
        for i in re.split('-|\n', Tel):
            if 'да' in i:
                return 1
        return 0
    except: 
        return np.NaN 

In [19]:
def getBal(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        Bal = re.split('Балкон:|Лифт', table)[1]
        for i in re.split('-|\n', Bal):
            if 'да' in i:
                return 1
        return 0
    except: 
        return np.NaN 

In [20]:
def getFloor(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        Floor = re.split('Этаж:|Тип дома', table)[1]
        for i in re.split('-|\n', Floor):
            if '\xa0' in i:
                a = i.replace(' ','')
                a = a.split('/',1)[0]
                return int(a.replace('\xa0',''))
        return np.NaN
    except: 
        return np.NaN 

In [21]:
def getNfloors(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        Nfloors = re.split('Этаж:|Тип дома', table)[1]
        for i in re.split('-|\n', Nfloors):
            if '\xa0' in i:
                a = i.replace(' ','')
                a = a.split('/',1)[1]
                return int(a.replace('\xa0',''))
        return np.NaN
    except: 
        return np.NaN 

In [22]:
# Эта функция нам пригодится дальше, она оставляет в списке только уникальные элементы

In [23]:
def remove_dublicates(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [24]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

# Цикл:

In [25]:
#Вот эта ссылка позволит нам задавать количество комнат, район и номер страницы
#С ее помощью можно выкачать почи весь Циан, так как по таким узким запросам, которые она дает, 
#почти всегда выпадает меньше 900 страниц.
#Но пока я ее не использую, тк на выкачку всего Циана уйдет очень много времени

In [26]:
mlink = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D={}&engine_version=2&offer_type=flat&p={}&room{}=1'

In [27]:
#Создем датафрейм с квартирами

In [28]:
FLATS = pd.DataFrame(columns=['n','rooms','price','totsp','livesp','kitsp','dist','metrdist','walk','brick','tel','bal','floor','nfloors','new'])

In [29]:
#Вместо той гибкой ссылки, будем использовать список вот таких больших ссылок
#Каждая дает ссылку на один округ, с возможностью гулять по страницам

In [30]:
districts = [\
'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',\
'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=23&district%5B1%5D=24&district%5B10%5D=33&district%5B11%5D=34&district%5B12%5D=35&district%5B13%5D=36&district%5B14%5D=37&district%5B15%5D=38&district%5B2%5D=25&district%5B3%5D=26&district%5B4%5D=27&district%5B5%5D=28&district%5B6%5D=29&district%5B7%5D=30&district%5B8%5D=31&district%5B9%5D=32&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',\
'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=39&district%5B1%5D=40&district%5B10%5D=49&district%5B11%5D=50&district%5B12%5D=51&district%5B13%5D=52&district%5B14%5D=53&district%5B15%5D=54&district%5B16%5D=55&district%5B2%5D=41&district%5B3%5D=42&district%5B4%5D=43&district%5B5%5D=44&district%5B6%5D=45&district%5B7%5D=46&district%5B8%5D=47&district%5B9%5D=48&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',\
'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=56&district%5B1%5D=57&district%5B10%5D=66&district%5B11%5D=67&district%5B12%5D=68&district%5B13%5D=69&district%5B14%5D=70&district%5B15%5D=71&district%5B2%5D=58&district%5B3%5D=59&district%5B4%5D=60&district%5B5%5D=61&district%5B6%5D=62&district%5B7%5D=63&district%5B8%5D=64&district%5B9%5D=65&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',\
'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=72&district%5B1%5D=73&district%5B10%5D=82&district%5B11%5D=83&district%5B2%5D=74&district%5B3%5D=75&district%5B4%5D=76&district%5B5%5D=77&district%5B6%5D=78&district%5B7%5D=79&district%5B8%5D=80&district%5B9%5D=81&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',\
'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=84&district%5B1%5D=85&district%5B10%5D=94&district%5B11%5D=95&district%5B12%5D=96&district%5B13%5D=97&district%5B14%5D=98&district%5B15%5D=99&district%5B2%5D=86&district%5B3%5D=87&district%5B4%5D=88&district%5B5%5D=89&district%5B6%5D=90&district%5B7%5D=91&district%5B8%5D=92&district%5B9%5D=93&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',\
'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=100&district%5B1%5D=101&district%5B10%5D=110&district%5B11%5D=111&district%5B2%5D=102&district%5B3%5D=103&district%5B4%5D=104&district%5B5%5D=105&district%5B6%5D=106&district%5B7%5D=107&district%5B8%5D=108&district%5B9%5D=109&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',\
'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=112&district%5B1%5D=113&district%5B10%5D=122&district%5B11%5D=123&district%5B12%5D=124&district%5B13%5D=348&district%5B14%5D=349&district%5B15%5D=350&district%5B2%5D=114&district%5B3%5D=115&district%5B4%5D=116&district%5B5%5D=117&district%5B6%5D=118&district%5B7%5D=119&district%5B8%5D=120&district%5B9%5D=121&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',\
'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=125&district%5B1%5D=126&district%5B2%5D=127&district%5B3%5D=128&district%5B4%5D=129&district%5B5%5D=130&district%5B6%5D=131&district%5B7%5D=132&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'\
            ]

In [31]:
#Теперь запускаем цикл, который идет по округам и по страничкам в округах.
#В каждом округе выкачиваем 30 страниц, и если случай мы вакачали две одинаковые квартиры (такое бывает!),
#то выкидываем повторяющиеся элементы с помощью заранее приготовленной функции.
#В каждый параметр записываем результат специально приготовленной для него функции.

In [32]:
f_link = "http://www.cian.ru/sale/flat/{}"

In [33]:
count = 0
for district in districts:
    
    links = []
    for page in range(1, 31):
        try:
            page_url =  district.format(page)

            search_page = requests.get(page_url)
            search_page = search_page.content
            search_page = BeautifulSoup(search_page, 'lxml')

            flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
            flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))


            for link in flat_urls:
                if link.isdigit():
                    links.append(link)
        except:
            print('!!!')
            
    links = remove_dublicates(links)
    
    for link in links:
        try:
            count = count + 1

            flat_page = requests.get(f_link.format(link))
            flat_page = flat_page.content
            flat_page = BeautifulSoup(flat_page, 'lxml')
            to_append = {'n':count,'rooms':getRoom(flat_page),\
                         'price':getPrice(flat_page),\
                         'totsp':getTotsp(flat_page),\
                         'livesp':getLivesp(flat_page),\
                         'kitsp':getKitsp(flat_page),\
                         'dist':getDist(flat_page),\
                         'metrdist':getMetrdist(flat_page),\
                         'walk':getWalk(flat_page),\
                         'brick':getBrick(flat_page),\
                         'tel':getTel(flat_page),\
                         'bal':getBal(flat_page),\
                         'floor':getFloor(flat_page),\
                         'nfloors':getNfloors(flat_page),\
                         'new':getNew(flat_page),\
                        }
            FLATS = FLATS.append(to_append, ignore_index=True)
        except:
            print('!!!')
        

In [34]:
#Вот массив с квартирами

In [35]:
FLATS.to_csv('C:/Users/Ivan/Desktop/FLATS.csv')